In [ ]:
import os
import pandas as pd
'''
读取文件夹中的所有.tif文件名，并将这些文件名保存到一个CSV文件中
'''

def save_tif_filenames_to_csv(images_dir, output_csv):
    # 获取指定文件夹中的所有 .tif 文件名
    image_files = [f for f in os.listdir(images_dir) if f.endswith('.tif')]
    
    # 创建一个DataFrame
    df = pd.DataFrame(image_files, columns=['filename'])
    
    # 将DataFrame保存到CSV文件
    df.to_csv(output_csv, index=False)
    print(f'已将 {len(image_files)} 个 .tif 文件名保存到 {output_csv}.')

# 示例用法
images_dir = r'H:\ArcGIS File\extracting-slums-from-satellite-imagery\6.2_training_data_rio_rio2019\labels[2019]'
output_csv = 'output_directory/output_filenames.csv'
save_tif_filenames_to_csv(images_dir, output_csv)

In [ ]:
import os
import rasterio
import matplotlib.pyplot as plt
from PIL import Image

import cv2
import numpy as np

from rasterio.enums import Resampling

In [ ]:
# 检查ArcGIS得到的tif的mask（是不是空，是0/1/2？）
 #发现问题：没有mask的图像可能也有一个1点，导致segmentation不是大于6的list(多于三个点的polygon)

import rasterio
import numpy as np

# 读取TIF文件路径
tif_path = r'G:\ArcGIS File\extracting-slums-from-satellite-imagery\3_training_data_roi_roiall2019_cell0.5_tile2048_stride1024\labels\000000005131.tif' #000000004515.tif,000000004516.tif
# tif_path = r'G:\ArcGIS File\extracting-slums-from-satellite-imagery\2_training_data_Tijuca2019_cell0.5_tile400_stride200\labels\000000012674.tif'  # or 739

# 使用rasterio打开tif文件
with rasterio.open(tif_path) as src:
    # 读取图像数据
    img_data = src.read(1)  # 读取第一波段的数据，通常是灰度图像

    unique_values = np.unique(img_data)  # 获取图像中的所有唯一值
    print("Unique values in the image:", unique_values)

    # 检查图像是否全为0
    if np.all(img_data == 0):
        print(f"{tif_path} is all zeros")
    else:
        ones_positions = np.argwhere(img_data == 1)  # 获取值为1的所有位置
        
        # 输出找到的位置和数量
        print(f"Found {len(ones_positions)} ones at the following positions:")
        print(ones_positions)


Unique values in the image: [0 1]
Found 5 ones at the following positions:
[[1410    0]
 [1411    0]
 [1412    0]
 [1413    0]
 [1414    0]]


In [ ]:
# for a folder
labels_dir = r'E:\ArcGIS File\extracting-slums-from-satellite-imagery\training_data_roi_roiall2019_cell0.5_tile2048_stride1024_in use\labels'
tif_files = [f for f in os.listdir(labels_dir) if f.endswith('.tif')]

# 存储所有唯一值
all_unique_values = set()

# 遍历所有TIF文件
for tif_file in tif_files:
    tif_path = os.path.join(labels_dir, tif_file)
    
    # 使用rasterio打开tif文件
    with rasterio.open(tif_path) as src:
        # 读取图像数据
        img_data = src.read(1)  # 读取第一波段的数据，通常是灰度图像
        print(f"{tif_file}:{np.unique(img_data)}")
        
        # 获取图像中的所有唯一值并加入到集合中
        all_unique_values.update(np.unique(img_data))

# 输出所有唯一值
print("All unique values across all TIF files:", sorted(all_unique_values))


000000001024.tif:[0 1]
000000001025.tif:[0 1]
000000001026.tif:[0 1]
000000001027.tif:[0 1]
000000001028.tif:[0 1]
000000001029.tif:[0 1]
000000001030.tif:[0 1]
000000001280.tif:[0 1]
000000001281.tif:[0 1]
000000001282.tif:[0 1]
000000001283.tif:[0 1]
000000001284.tif:[0 1]
000000001285.tif:[0 1]
000000001286.tif:[0 1]
000000001287.tif:[0 1]
000000001288.tif:[0 1]
000000001289.tif:[0 1]
000000001290.tif:[0 1]
000000001291.tif:[0 1]
000000001292.tif:[0 1]
000000001293.tif:[0 1]
000000001294.tif:[0 1]
000000001295.tif:[0 1]
000000001296.tif:[0 1]
000000001297.tif:[0 1]
000000001298.tif:[0 1]
000000001299.tif:[0 1]
000000001300.tif:[0 1]
000000001301.tif:[0 1]
000000001302.tif:[0 1]
000000001303.tif:[0 1]
000000001304.tif:[0 1]
000000001305.tif:[0 1]
000000001306.tif:[0 1]
000000001307.tif:[0 1]
000000001308.tif:[0 1]
000000001309.tif:[0 1]
000000001310.tif:[0 1]
000000001311.tif:[0 1]
000000001312.tif:[0 1]
000000001313.tif:[0 1]
000000001314.tif:[0 1]
000000001315.tif:[0 1]
00000000131

In [ ]:
import json

# 加载 COCO 数据集 JSON 文件
json_file = 'formatted_data.json'  # 请替换为您的 JSON 文件路径

with open(json_file, 'r') as f:
    data = json.load(f)

# 遍历每个注释，检查 segmentation 和 bbox 字段
for ann in data['annotations']:
    # 检查是否有 segmentation 和 bbox
    segmentation = ann.get('segmentation', [])
    bbox = ann.get('bbox', [])

    # 打印信息来检查是否缺失或无效
    if not segmentation:
        print(f"Missing or empty segmentation for annotation id: {ann['id']}")
    if not bbox:
        print(f"Missing or empty bbox for annotation id: {ann['id']}")

    # 可选：检查 segmentation 和 bbox 是否格式正确
    if len(bbox) != 4:
        print(f"Invalid bbox format for annotation id: {ann['id']}, bbox: {bbox}")

    if segmentation and isinstance(segmentation[0], list):
        for seg in segmentation:
            if len(seg) < 6:  # segmentation 至少应该有 3 个点（x, y 坐标）
                print(f"Invalid segmentation format for annotation id: {ann['id']}, segmentation: {seg}")

# 如果没有错误或警告，打印成功
print("Finished checking annotations.")


Invalid segmentation format for annotation id: 158, segmentation: [324, 399, 332, 399]
Invalid segmentation format for annotation id: 159, segmentation: [324, 399, 332, 399]
Invalid segmentation format for annotation id: 160, segmentation: [124, 399, 132, 399]
Invalid segmentation format for annotation id: 161, segmentation: [124, 399, 132, 399]
Invalid segmentation format for annotation id: 543, segmentation: [262, 399, 263, 399]
Invalid segmentation format for annotation id: 544, segmentation: [262, 399, 263, 399]
Invalid segmentation format for annotation id: 545, segmentation: [262, 399, 263, 399]
Invalid segmentation format for annotation id: 546, segmentation: [262, 399, 263, 399]
Invalid segmentation format for annotation id: 547, segmentation: [262, 399, 263, 399]
Invalid segmentation format for annotation id: 550, segmentation: [62, 399, 63, 399]
Invalid segmentation format for annotation id: 551, segmentation: [62, 399, 63, 399]
Invalid segmentation format for annotation id: 

In [ ]:
# beautify json

# 读取JSON文件
with open("coco_format.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# 格式化JSON数据
formatted_json = json.dumps(data, indent=4, ensure_ascii=False)

# 保存格式化后的JSON到新文件
with open("formatted_data.json", "w", encoding="utf-8") as file:
    file.write(formatted_json)

print("JSON数据已格式化并保存到文件 'formatted_data.json'")

JSON数据已格式化并保存到文件 'formatted_data.json'


In [ ]:
import json

coco_format = {
    "images": [],
    "annotations": [],
    "categories": []
}

# 定义类别（假设只有一个类别）
category = {
    "id": 1,
    "name": "slum",
    "supercategory": "none"
}
coco_format["categories"].append(category)

annotation_id = 1
image_id = 1


In [ ]:
label_path = r'F:\ArcGIS File\extracting-slums-from-satellite-imagery\training_data_Tijuca2019_cell0.5_tile400_stride200\labels\000000012674.tif'
#739

with rasterio.open(label_path) as src:
    label_data = src.read(1)

# 假设每个标签图像只有一个对象
# 这里您可以根据需要提取更多详细的标注信息
# 确保标签数据是 uint8 格式
label_data = label_data.astype(np.uint8)

# 进行二值化处理，假设非零区域是对象
_, binary_mask = cv2.threshold(label_data, 0, 255, cv2.THRESH_BINARY) 
contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

segmentation = []

for contour in contours:
    print(len(contour))
    x, y, w, h = cv2.boundingRect(contour)
    area = cv2.contourArea(contour)
    segmentation.append(contour.flatten().tolist())

1
